In [4]:
import pandas as pd
import os
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import plotly.express as px
import plotly
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
print('Inicializado Rotina json_collect.py.')
os.chdir(sys.path[0])
# os.chdir(os.path.join(os.getcwd(),"json_collect.py"))                  # Modifica o diretório de trabalho, para os caminhos relativos funcionar. (https://stackoverflow.com/questions/1432924/python-change-the-scripts-working-directory-to-the-scripts-own-directory)
# print(os.getcwd())
%run ./json_collect.py                      

Inicializado Rotina json_collect.py.


In [6]:
# sites: https://dadosabertos.bcb.gov.br/dataset/expectativas-mercado
# Sintase: https://dadosabertos.bcb.gov.br/dataset/expectativas-mercado/resource/d420a704-75a7-4f45-8f4b-0fca813c70f0
# documentação: https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/documentacao

In [7]:
# Relações datas das reunições inseridas manualmente quando não tem previsão
df_selic_data=pd.read_csv('./input/selic_datas.csv',index_col='Reuniao')
# Dados baixados pelo .json
df_selic=pd.read_csv('./output/selic_data_df.csv',index_col='Reuniao')
df_selic=df_selic.drop(columns=['Unnamed: 0','Indicador'])
df_selic['Data']=df_selic.apply(lambda x: datetime.datetime.strptime(x.Data,'%Y-%m-%d').date(),axis=1)

# Relaciona a reunião com sua Data.
df_selic['data_reuniao']=''
for i in range(0, len(df_selic)):
    busca_data=''
    try:
        busca_data=df_selic_data.loc[df_selic.index[i],'data_reuniao']
        busca_data=datetime.datetime.strptime(busca_data,'%Y-%m-%d').date()
        df_selic.loc[df_selic.index[i],'data_reuniao']=busca_data

    except:
        #### será definida um expectativa para a data.
        # busca_data=df_selic_data.loc[df_selic.index[i],'data_reuniao']
        # df_selic.loc[df_selic.index[i],'data_reuniao']=busca_data
        continue

# df_selic['data_reuniao']=df_selic.apply(lambda x: datetime.datetime.strptime(x.data_reuniao,'%Y-%m-%d').date(),axis=1)
# df_selic=df_selic[df_selic['data_reuniao']!='']
df_selic=df_selic.sort_values(by=['Data','data_reuniao','baseCalculo'],ascending=[False,True,False])

'''Base de cálculo para as estatísticas
 baseada no prazo de validade das expectativas
 informadas pelas instituições informantes:
  - 0: uso das expectativas mais recentes informadas
   pelas instituições participantes a partir do
   30º dia anterior à data de cálculo das estatísticas 
  - 1: uso das expectativas mais recentes informadas
   pelas instituições participantes a partir do
    4º dia útil anterior à data de cálculo das estatísticas'''

df_selic.head(20)

,Data,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo,data_reuniao
Reuniao,,,,,,,,,
R7/2022,2022-10-14,13.75,13.75,0.03,13.75,14.00,58,1,2022-10-26
R7/2022,2022-10-14,13.76,13.75,0.05,13.75,14.00,128,0,2022-10-26
R8/2022,2022-10-14,13.75,13.75,0.03,13.75,14.00,58,1,2022-12-07
R8/2022,2022-10-14,13.76,13.75,0.05,13.75,14.00,131,0,2022-12-07
R1/2023,2022-10-14,13.75,13.75,0.06,13.50,14.00,57,1,2023-02-01
R1/2023,2022-10-14,13.75,13.75,0.07,13.25,14.00,127,0,2023-02-01
R2/2023,2022-10-14,13.72,13.75,0.12,13.25,14.00,57,1,2023-03-22
R2/2023,2022-10-14,13.73,13.75,0.13,13.25,14.00,127,0,2023-03-22
R3/2023,2022-10-14,13.59,13.75,0.31,12.50,14.00,57,1,2023-05-03


In [8]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_selic,x='data_reuniao',y='Mediana', color='Data')
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
fig.show()

In [9]:
# Informada à partir do 4º dia útil anterior à data de cálculo da expectativa.
df_selic=df_selic[df_selic['baseCalculo']==1]           
# Expectativa mais atual
df_selic_ultima=df_selic[df_selic['Data']==df_selic['Data'].max()]
# df_selic_ultima['mediana_mensal']=df_selic_ultima.apply(lambda x: (1+(x.Mediana/100))**(1/12)-1,axis=1)

In [10]:
# Insere coluna com números de dias no período (entre duas datas)
df_selic_ultima.loc[df_selic_ultima.index[0],'periodo']=(df_selic_ultima.loc[df_selic_ultima.index[0],'data_reuniao']-datetime.datetime.today().date()).days
# Define número índice de ref. 100 na data de hoje.
selic_atual=''
while type(selic_atual)!=float:
    try:
        selic_atual=float(input('Qual é a SELIC atual?(\d\d.d\d\)'))
    except:
        continue

C:\Users\bruno.bfn\AppData\Local\Temp\ipykernel_2276\1858555858.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
# Cria num_indice inicial atribuindo 100 para a data de hoje
## Pode acontecer de demora para atualização do json do bc, gerando o primeiro período negativo, mas não tem problema
df_selic_ultima.loc[df_selic_ultima.index[0],'num_indice']=100*(1+selic_atual/100)**(df_selic_ultima.loc[df_selic_ultima.index[0],'periodo']/365)

# num_indice_f=num_indice_i*(1+taxa)**(periodo)
for i in range(1,len(df_selic_ultima)):
    df_selic_ultima.loc[df_selic_ultima.index[i],'periodo']=(df_selic_ultima.loc[df_selic_ultima.index[i],'data_reuniao']-df_selic_ultima.loc[df_selic_ultima.index[i-1],'data_reuniao']).days
    df_selic_ultima.loc[df_selic_ultima.index[i],'num_indice']=df_selic_ultima.loc[df_selic_ultima.index[i-1],'num_indice']*(1+df_selic_ultima.loc[df_selic_ultima.index[i-1],'Mediana']/100)**(df_selic_ultima.loc[df_selic_ultima.index[i],'periodo']/365)
df_selic_ultima.to_csv('./output/selic_ultima_expectativa.csv',float_format='%.2f')


C:\Users\bruno.bfn\AppData\Local\Temp\ipykernel_2276\2648126627.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
df_selic_ultima

,Data,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo,data_reuniao,periodo,num_indice
Reuniao,,,,,,,,,,,
R7/2022,2022-10-14,13.75,13.75,0.03,13.75,14.00,58,1,2022-10-26,7.00,100.25
R8/2022,2022-10-14,13.75,13.75,0.03,13.75,14.00,58,1,2022-12-07,42.00,101.74
R1/2023,2022-10-14,13.75,13.75,0.06,13.50,14.00,57,1,2023-02-01,56.00,103.78
R2/2023,2022-10-14,13.72,13.75,0.12,13.25,14.00,57,1,2023-03-22,49.00,105.59
R3/2023,2022-10-14,13.59,13.75,0.31,12.50,14.00,57,1,2023-05-03,42.00,107.16
R4/2023,2022-10-14,13.27,13.25,0.54,11.50,14.00,57,1,2023-06-21,49.00,109.03
R5/2023,2022-10-14,12.76,12.75,0.70,10.50,14.00,57,1,2023-08-02,42.00,110.60
R6/2023,2022-10-14,12.20,12.25,0.77,10.00,13.75,57,1,2023-09-20,49.00,112.40
R7/2023,2022-10-14,11.63,11.75,0.82,9.00,13.75,57,1,2023-11-01,42.00,113.91


In [13]:
#### Função que calcula a SELIC do período entre data atual e data final
def calc_selic_periodo(df,data_f):
    # DF é um dataframe que contenha a culuna com num_indice
    
    # Utiliza a taxa digitada real vigente
    if data_f<=df.loc[df.index[0],'data_reuniao']:
        n_dias=(data_f-datetime.datetime.today().date()).days
        return 100*(1+selic_atual/100)**(n_dias/365)
    
    # Utiliza o número índice anterior + variação dos dias dentro do período
    else:
        # Último número índice
        df=df[df['data_reuniao']<=data_f]
        n_indice=df.loc[df.index[-1],'num_indice']
        taxa=df.loc[df.index[-1],'Mediana']
        n_dias=(data_f-df.loc[df.index[-1],'data_reuniao']).days
        return n_indice*(1+taxa/100)**(n_dias/365)


In [14]:
calc_selic_periodo(df_selic_ultima,datetime.datetime(2022,10,25).date())

100.21200447567182

In [15]:
df_monthly=pd.read_csv('./output/monthly_data_df.csv')
df_monthly=df_monthly.drop(columns=['Unnamed: 0'])

# Parses
df_monthly['Data']=df_monthly.apply(lambda x: datetime.datetime.strptime(x.Data,'%Y-%m-%d').date(),axis=1)

df_monthly['DataReferencia']=df_monthly.apply(lambda x: datetime.datetime.strptime(x.DataReferencia,'%m/%Y').date(),axis=1)
delta = relativedelta(months=1)
delta_d = timedelta(days=1)
df_monthly['DataReferencia']=df_monthly.apply(lambda x: x.DataReferencia+delta-delta_d,axis=1)

df_monthly=df_monthly.sort_values(by=['Data','DataReferencia','numeroRespondentes'],ascending=[False,True,False])
# df_monthly=df_selic[df_selic['baseCalculo']==1]           # não sei qual é a diferença da base de cálculo.
df_monthly.head(20)

,Indicador,Data,DataReferencia,Media,Mediana,Minimo,Maximo,numeroRespondentes
419,Taxa de desocupação,2022-10-14,2022-09-30,8.84,8.80,8.30,9.87,54
418,Taxa de desocupação,2022-10-14,2022-09-30,8.81,8.80,8.30,9.87,23
403,IPCA,2022-10-14,2022-10-31,0.33,0.33,0.00,0.69,137
405,Câmbio,2022-10-14,2022-10-31,5.26,5.25,4.90,5.50,111
417,IPCA Administrados,2022-10-14,2022-10-31,-0.13,-0.12,-1.05,0.70,79
401,IGP-M,2022-10-14,2022-10-31,-0.11,-0.09,-1.05,0.69,76
402,IPCA,2022-10-14,2022-10-31,0.33,0.34,0.00,0.65,75
409,IPCA Livres,2022-10-14,2022-10-31,0.49,0.48,0.20,0.80,75
415,IPCA Alimentação no domicílio,2022-10-14,2022-10-31,0.17,0.13,-0.34,0.82,69
411,IPCA Serviços,2022-10-14,2022-10-31,0.61,0.60,0.22,0.97,68


In [16]:
df_monthly['Indicador'].unique()

array(['Taxa de desocupação', 'IPCA', 'Câmbio', 'IPCA Administrados',
       'IGP-M', 'IPCA Livres', 'IPCA Alimentação no domicílio',
       'IPCA Serviços', 'IPCA Bens industrializados'], dtype=object)

In [17]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='IPCA'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='IGP-M'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/IGPM_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='Câmbio'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/cambio_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='Taxa de desocupação'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/tx_desocupacao_forcast.html')
# fig.show()

'./output/tx_desocupacao_forcast.html'

In [18]:
# Expectativa mais atual para IPCA
df_ipca=df_monthly[df_monthly['Indicador']=='IPCA']

data_previsao=df_ipca['Data'].max()
df_ipca_ultima=df_ipca[df_ipca['Data']==data_previsao]
df_ipca_ultima=df_ipca_ultima.groupby(['DataReferencia']).mean() # adotado média 

df_ipca_ultima['cumsum_mediana'] = df_ipca_ultima['Mediana'].cumsum()
#cumsum()-> soma acumulada de uma lista ou array termo a termo
#groupby -> delimita em grupos separados para análise
# df['Adjusted Quantity'] = df.groupby('ação_ref')['Adjusted Quantity'].cumsum()

In [19]:
# Insere coluna com números de dias no período (entre duas datas)
df_ipca_ultima.loc[df_ipca_ultima.index[0],'periodo']=(df_ipca_ultima.index[0]-datetime.datetime.today().date()).days

# Define número índice de ref. 100 na data de hoje.
df_ipca_ultima.loc[df_ipca_ultima.index[0],'num_indice']=100*(1+df_ipca_ultima.loc[df_ipca_ultima.index[0],'Mediana']/100)**(df_ipca_ultima.loc[df_ipca_ultima.index[0],'periodo']/30)
for i in range(1,len(df_ipca_ultima)):
    df_ipca_ultima.loc[df_ipca_ultima.index[i],'periodo']=(df_ipca_ultima.index[i]-df_ipca_ultima.index[i-1]).days
    df_ipca_ultima.loc[df_ipca_ultima.index[i],'num_indice']=df_ipca_ultima.loc[df_ipca_ultima.index[i-1],'num_indice']*(1+df_ipca_ultima.loc[df_ipca_ultima.index[i],'Mediana']/100)**(df_ipca_ultima.loc[df_ipca_ultima.index[i],'periodo']/30)
df_ipca_ultima.to_csv('./output/ipca_ultima_expectativa.csv',float_format='%.4f')
df_ipca_ultima

,Media,Mediana,Minimo,Maximo,numeroRespondentes,cumsum_mediana,periodo,num_indice
DataReferencia,,,,,,,,
2022-10-31,0.33,0.34,0.00,0.67,106.00,0.34,12.00,100.13
2022-11-30,0.42,0.43,0.00,0.90,106.00,0.77,30.00,100.57
2022-12-31,0.68,0.69,-0.16,1.20,106.00,1.46,31.00,101.28
2023-01-31,0.57,0.55,0.20,1.37,101.00,2.00,31.00,101.86
2023-02-28,0.61,0.61,0.20,0.99,101.00,2.62,28.00,102.44
2023-03-31,0.45,0.43,0.12,1.21,101.00,3.05,31.00,102.90
2023-04-30,0.45,0.43,0.15,0.85,101.00,3.49,30.00,103.35
2023-05-31,0.33,0.34,0.14,0.67,101.00,3.82,31.00,103.71
2023-06-30,0.32,0.32,-0.04,0.54,101.00,4.14,30.00,104.04


In [20]:
# from matplotlib.pyplot import figure
# figure(figsize=(10, 5), dpi=120)
# plt.plot(df_ipca_ultima.index, df_ipca_ultima['Mediana'], label=f'ipca_expectaiva_{data_previsao}')
# plt.xticks(rotation=90)
# plt.legend()
# plt.show()


In [21]:
#### Função que calcula a IPCA do período entre data hoje e data final
def calc_ipca_periodo(df,data_f):
    # DF é um dataframe que contenha a coluna com num_indice
    
    # Utiliza a taxa digitada real vigente
    if data_f<=df.index[0]:
        # print('primeiro mês')
        n_dias=(data_f-datetime.datetime.today().date()).days
        # print(n_dias)
        return 100*(1+df.loc[df.index[0],'Mediana']/100)**(n_dias/30)
    
    # Não calcula num_indice para o último termo, pois já está tabelado. 
    elif data_f==df.index[-1]:
        return df.loc[df.index[-1],'num_indice']

    # Utiliza o número índice anterior + variação dos dias dentro do período
    else:
        # Último número índice
        df_anterior=df[df.index<=data_f]
        # print(df_anterior)
        n_indice=df_anterior.loc[df_anterior.index[-1],'num_indice']
        #busca o 1º termo depois do corte da df_anterior
        taxa=df.loc[df.index[len(df_anterior)],'Mediana']    
        # print(f'taxa:{taxa}')
        n_dias=(data_f-df_anterior.index[-1]).days
        # print(f'n_dias:{n_dias}')
        return n_indice*(1+taxa/100)**(n_dias/30)

In [22]:
calc_ipca_periodo(df_ipca_ultima,datetime.datetime(2022,11,30).date())

100.56944788531762

In [23]:
df_ipca_ultima

,Media,Mediana,Minimo,Maximo,numeroRespondentes,cumsum_mediana,periodo,num_indice
DataReferencia,,,,,,,,
2022-10-31,0.33,0.34,0.00,0.67,106.00,0.34,12.00,100.13
2022-11-30,0.42,0.43,0.00,0.90,106.00,0.77,30.00,100.57
2022-12-31,0.68,0.69,-0.16,1.20,106.00,1.46,31.00,101.28
2023-01-31,0.57,0.55,0.20,1.37,101.00,2.00,31.00,101.86
2023-02-28,0.61,0.61,0.20,0.99,101.00,2.62,28.00,102.44
2023-03-31,0.45,0.43,0.12,1.21,101.00,3.05,31.00,102.90
2023-04-30,0.45,0.43,0.15,0.85,101.00,3.49,30.00,103.35
2023-05-31,0.33,0.34,0.14,0.67,101.00,3.82,31.00,103.71
2023-06-30,0.32,0.32,-0.04,0.54,101.00,4.14,30.00,104.04


In [24]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_ipca_ultima,x=df_ipca_ultima.index,y=['cumsum_mediana'])
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/tx_desocupacao_forcast.html')
fig.show()

In [25]:
# Relação de datas até o mínimo entre os dataframes, para evitar erros
lista_datas_todos_dias=pd.date_range(start=datetime.datetime.today().date(),end=min(df_selic_ultima['data_reuniao'].max(),df_ipca_ultima.index.max()))
df_juros_real=pd.DataFrame(index=lista_datas_todos_dias,columns=['num_indice_selic','num_indice_ipca'],dtype='float64')
for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],'num_indice_selic']=calc_selic_periodo(df_selic_ultima,df_juros_real.index[i].date())
    df_juros_real.loc[df_juros_real.index[i],'num_indice_ipca']=calc_ipca_periodo(df_ipca_ultima,df_juros_real.index[i].date())
df_juros_real

,num_indice_selic,num_indice_ipca
2022-10-19,100.00,100.00
2022-10-20,100.04,100.01
2022-10-21,100.07,100.02
2022-10-22,100.11,100.03
2022-10-23,100.14,100.04
...,...,...
2024-10-05,124.78,109.02
2024-10-06,124.81,109.03
2024-10-07,124.84,109.04
2024-10-08,124.87,109.05


In [26]:
df_ipca_ultima

,Media,Mediana,Minimo,Maximo,numeroRespondentes,cumsum_mediana,periodo,num_indice
DataReferencia,,,,,,,,
2022-10-31,0.33,0.34,0.00,0.67,106.00,0.34,12.00,100.13
2022-11-30,0.42,0.43,0.00,0.90,106.00,0.77,30.00,100.57
2022-12-31,0.68,0.69,-0.16,1.20,106.00,1.46,31.00,101.28
2023-01-31,0.57,0.55,0.20,1.37,101.00,2.00,31.00,101.86
2023-02-28,0.61,0.61,0.20,0.99,101.00,2.62,28.00,102.44
2023-03-31,0.45,0.43,0.12,1.21,101.00,3.05,31.00,102.90
2023-04-30,0.45,0.43,0.15,0.85,101.00,3.49,30.00,103.35
2023-05-31,0.33,0.34,0.14,0.67,101.00,3.82,31.00,103.71
2023-06-30,0.32,0.32,-0.04,0.54,101.00,4.14,30.00,104.04


In [27]:
# Não utilizada
def area(tipo,date):
    #tipo={selic,ipca}
    area=0
    df=df_juros_real[df_juros_real.index<=date]
    if (tipo=='selic' or tipo=='ipca'):
        for i in range(0,len(df)):
            area=area+1*df.loc[df.index[i],f'num_indice_{tipo}']
                
        return area
    else:
        print('Erro na escolha do tipo (selic ou ipca)')
    return 0

In [28]:
# df_juros_real['area_selic']=0
# for i in range(0,len(df_juros_real)):
#     # print(area('selic',df_juros_real.index[i]))
#     df_juros_real.loc[df_juros_real.index[i], 'area_selic']=area('selic',df_juros_real.index[i])
#     df_juros_real.loc[df_juros_real.index[i], 'area_ipca']=area('ipca',df_juros_real.index[i])

# df_juros_real

In [29]:
df_juros_real['dif_num_indice']=df_juros_real.apply(lambda x: x.num_indice_selic-x.num_indice_ipca+100,axis=1)
# df_juros_real['dif_area']=df_juros_real.apply(lambda x: x.area_selic-x.area_ipca,axis=1)
# df_juros_real['dif_area_por_dia']=df_juros_real.apply(lambda x: x.dif_area/(x.index-))
df_juros_real.to_csv('./output/num_indice_selic_ipca_diferenca.csv',float_format='%.2f')
df_juros_real


,num_indice_selic,num_indice_ipca,dif_num_indice
2022-10-19,100.00,100.00,100.00
2022-10-20,100.04,100.01,100.02
2022-10-21,100.07,100.02,100.05
2022-10-22,100.11,100.03,100.07
2022-10-23,100.14,100.04,100.10
...,...,...,...
2024-10-05,124.78,109.02,115.76
2024-10-06,124.81,109.03,115.78
2024-10-07,124.84,109.04,115.80
2024-10-08,124.87,109.05,115.82


In [30]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_juros_real,x=df_juros_real.index,y=['num_indice_selic','num_indice_ipca','dif_num_indice'])
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
fig.show()

In [31]:
from time import strptime
from pyxirr import xirr
# calculo da IRR
def cal_irr(x,date,tipo):                #DF refer^rencia e a data de análise

    if date==x.index[0]:
        return (0,0,0,0)
    
    dates=[x.index[0],date]
    
    # tipo pode assumir ('selic' ou 'pre')
    if tipo=='selic':
        amounts=[-x.loc[x.index[0],'dif_num_indice'],x.loc[date,'dif_num_indice']]
    elif tipo=='pre':
        amounts=[-x.loc[x.index[0],'dif_num_indice_pre'],x.loc[date,'dif_num_indice_pre']]
            
    else:
        print('Erro de tipo diferente de "selic" ou "pre"') 
    # date=datetime.datetime.strptime(date,'%Y-%m-%d').date()
    # x=x[x.index==x.index[0] | x.index.date()==date]
    # dates=x.index
    # amounts=x['dif_num_indice']
    t_aa=xirr(dates, amounts)
    t_am=(1+t_aa)**(1/12)-1
    # print('TII a.a.(%): ',t_aa*100)
    # print('TII a.m.(%): ',t_am*100)
    # print('Soma dos valores: ',-x['Valor'].sum())
    # print(dates)
    # print(amounts)
    return (t_aa*100,t_am*100,dates,amounts)


In [32]:
for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],'juros_real_acum_from_now']=cal_irr(df_juros_real,df_juros_real.index[i],'selic')[0]
    df_juros_real.loc[df_juros_real.index[i],'juros_real_am']=cal_irr(df_juros_real,df_juros_real.index[i],'selic')[1]

df_juros_real.head(30)

,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am
2022-10-19,100.00,100.00,100.00,0.00,0.00
2022-10-20,100.04,100.01,100.02,9.22,0.74
2022-10-21,100.07,100.02,100.05,9.22,0.74
2022-10-22,100.11,100.03,100.07,9.22,0.74
2022-10-23,100.14,100.04,100.10,9.22,0.74
2022-10-24,100.18,100.06,100.12,9.22,0.74
2022-10-25,100.21,100.07,100.15,9.22,0.74
2022-10-26,100.25,100.08,100.17,9.22,0.74
2022-10-27,100.28,100.09,100.19,9.22,0.74
2022-10-28,100.32,100.10,100.22,9.22,0.74


In [33]:
for i in range(0,len(df_juros_real)):
    # Primeiro dia do mês
    inicio_mes=df_juros_real.index[i]-timedelta(df_juros_real.index[i].day-1)
    if inicio_mes<=df_juros_real.index[0]:
        inicio_mes=df_juros_real.index[0]
    # print(f'Início do mês: {inicio_mes} - até: {df_juros_real.index[i]}')
    # Slice do DF do primeiro dia do mês até o dia da análise
    try:
        df_juros_real_i=df_juros_real[df_juros_real.index>=inicio_mes]
        df_juros_real_i=df_juros_real_i[df_juros_real_i.index<=df_juros_real.index[i]]
        # print(df_juros_real_i)
        df_juros_real.loc[df_juros_real.index[i],'juros_real_mesal']=cal_irr(df_juros_real_i,df_juros_real.index[i],'selic')[0]
    except:
        continue
df_juros_real.head(30)

,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am,juros_real_mesal
2022-10-19,100.00,100.00,100.00,0.00,0.00,0.00
2022-10-20,100.04,100.01,100.02,9.22,0.74,9.22
2022-10-21,100.07,100.02,100.05,9.22,0.74,9.22
2022-10-22,100.11,100.03,100.07,9.22,0.74,9.22
2022-10-23,100.14,100.04,100.10,9.22,0.74,9.22
2022-10-24,100.18,100.06,100.12,9.22,0.74,9.22
2022-10-25,100.21,100.07,100.15,9.22,0.74,9.22
2022-10-26,100.25,100.08,100.17,9.22,0.74,9.22
2022-10-27,100.28,100.09,100.19,9.22,0.74,9.22
2022-10-28,100.32,100.10,100.22,9.22,0.74,9.22


In [34]:
def calc_pre_periodo(data_f,taxa_pre):
    n_dias=(data_f-datetime.datetime.today().date()).days
    return 100*(1+taxa_pre/100)**(n_dias/365)

In [35]:
calc_pre_periodo(datetime.datetime(2022,10,29).date(),12)

100.31097206684598

In [36]:
# Gera num_indice_taxa_pre-fixa
taxa_pre=11.70
for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],f'num_indice_pre']=calc_pre_periodo(df_juros_real.index[i].date(),taxa_pre)
df_juros_real


,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am,juros_real_mesal,num_indice_pre
2022-10-19,100.00,100.00,100.00,0.00,0.00,0.00,100.00
2022-10-20,100.04,100.01,100.02,9.22,0.74,9.22,100.03
2022-10-21,100.07,100.02,100.05,9.22,0.74,9.22,100.06
2022-10-22,100.11,100.03,100.07,9.22,0.74,9.22,100.09
2022-10-23,100.14,100.04,100.10,9.22,0.74,9.22,100.12
...,...,...,...,...,...,...,...
2024-10-05,124.78,109.02,115.76,7.74,0.62,5.97,124.28
2024-10-06,124.81,109.03,115.78,7.73,0.62,5.97,124.32
2024-10-07,124.84,109.04,115.80,7.73,0.62,5.97,124.35
2024-10-08,124.87,109.05,115.82,7.73,0.62,5.97,124.39


In [37]:
# Agora que tenho num_indice_pre -> realizando os cálculos análogamente ao num_indice_selic
df_juros_real['dif_num_indice_pre']=df_juros_real.apply(lambda x: x.num_indice_pre-x.num_indice_ipca+100,axis=1)

for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],f'juros_real_acum_from_now_pre_{taxa_pre}%']=cal_irr(df_juros_real,df_juros_real.index[i],'pre')[0]

df_juros_real

,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am,juros_real_mesal,num_indice_pre,dif_num_indice_pre,juros_real_acum_from_now_pre_11.7%
2022-10-19,100.00,100.00,100.00,0.00,0.00,0.00,100.00,100.00,0.00
2022-10-20,100.04,100.01,100.02,9.22,0.74,9.22,100.03,100.02,7.25
2022-10-21,100.07,100.02,100.05,9.22,0.74,9.22,100.06,100.04,7.25
2022-10-22,100.11,100.03,100.07,9.22,0.74,9.22,100.09,100.06,7.25
2022-10-23,100.14,100.04,100.10,9.22,0.74,9.22,100.12,100.08,7.25
...,...,...,...,...,...,...,...,...,...
2024-10-05,124.78,109.02,115.76,7.74,0.62,5.97,124.28,115.26,7.50
2024-10-06,124.81,109.03,115.78,7.73,0.62,5.97,124.32,115.29,7.50
2024-10-07,124.84,109.04,115.80,7.73,0.62,5.97,124.35,115.31,7.50
2024-10-08,124.87,109.05,115.82,7.73,0.62,5.97,124.39,115.34,7.50


In [38]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_juros_real,x=df_juros_real.index,y=['juros_real_acum_from_now','juros_real_mesal',f'juros_real_acum_from_now_pre_{taxa_pre}%'])
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/Juros_real_forcast.html')
fig.show()
